In [1]:
import pandas as  pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data_path = 'processed_courses.csv'
data = pd.read_csv(data_path)
data.head(2)

,Unnamed: 0.1,Unnamed: 0,title,enrolled,rating,Instructor,Organization,Description,Modules/Courses,Level,URL
0,0,0,Analytical Solutions to Common Healthcare Prob...,"5,710",4.6,Brian Paciotti,"University of California, Davis","In this course, we’re going to go over analyti...",4 modules,Intermediate level,https://www.coursera.org/learn/analytical-solu...
1,1,1,Understanding Einstein: The Special Theory of ...,"170,608",4.9,Larry Randles Lagerstrom,Stanford University,In this course we will seek to “understand Ein...,8 modules,Beginner level,https://www.coursera.org/learn/einstein-relati...


In [46]:
data.isnull().sum()

Unnamed: 0.1       0
Unnamed: 0         0
title              0
enrolled           0
rating             0
Instructor         0
Organization       0
Description        0
Modules/Courses    0
Level              0
URL                0
dtype: int64

In [3]:
df = data['title']
df = np.array(df)

In [4]:
ddf = data['Description']
ddf = np.array(ddf)

In [6]:
ddf.shape

(5861,)

In [7]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Deepraj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Deepraj\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
lematizer = WordNetLemmatizer()

In [9]:
corpus = []
for sent in df:
    sentance = sent.lower()
    lemmas = [lematizer.lemmatize(word) for word  in word_tokenize(sentance) if word.isalpha() and word not in stop]
    filtered = ' '.join(lemmas)
    corpus.append(filtered)

In [10]:
desc = []
for sent in ddf:
    sentance = sent.lower()
    lemmas = [lematizer.lemmatize(word) for word  in word_tokenize(sentance) if word.isalpha() and word not in stop]
    filtered = ' '.join(lemmas)
    desc.append(filtered)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [12]:
bow = CountVectorizer(ngram_range=(1,2))
corpus_bow = bow.fit_transform(corpus)

In [13]:
tfidf = TfidfVectorizer(ngram_range=(1,2))
corpus_tfidf = tfidf.fit_transform(corpus)

In [14]:
desc_tfidf = TfidfVectorizer(ngram_range=(1,2))
desc_vec = desc_tfidf.fit_transform(desc)

In [15]:
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
def vectorize(title):
    return  desc_tfidf.transform([title])

In [18]:
def get_courses(titles):
    courses = []
    for title in titles:
        title_tfidf = vectorize(title)
        similar = cosine_similarity(corpus_tfidf,title_tfidf).flatten()
        top5 = similar.argsort()[::-1][:10]
        courses.append(top5)
        return courses    

In [28]:
def get_courses_by_desc(titles):
    titles = ','.join(titles)
    courses_by_dec = []
    for title in titles:
        title_tfidf = vectorize(title)
        similar = cosine_similarity(desc_vec,title_tfidf).flatten()
        top5 = similar.argsort()[::-1][:5]
        courses_by_dec.append(top5)
        return courses_by_dec
    

In [29]:
get_courses_by_desc(['Data Science'])

[array([5860, 1945, 1947, 1948, 1949], dtype=int64)]

In [31]:
from utils import save_file
import os
path = os.path.join('Artifacts', 'vec.pkl')
save_file(file_path=path,obj=desc_tfidf)

In [32]:
d = data
d.iloc[3869,:].to_dict

<bound method Series.to_dict of Unnamed: 0.1                                                    4378
Unnamed: 0                                                      4378
title                                          What is Data Science?
enrolled                                                   1,057,144
rating                                                           4.7
Instructor                                                 Rav Ahuja
Organization                                                     IBM
Description        Do you want to know why data science has been ...
Modules/Courses                                            4 modules
Level                                                 Beginner level
URL                https://www.coursera.org/learn/what-is-datasci...
Name: 3869, dtype: object>

In [47]:
skill = ['hibernate', 'junit', 'maven', 'rest apis', 'spring boot']
m = tfidf.transform(skill)
course = get_courses_by_desc(skill)

In [48]:
course

[array([5860, 1945, 1947, 1948, 1949], dtype=int64)]

In [61]:
a = []
x = d.iloc[5860,:].to_dict()
y = d.iloc[1945,:].to_dict()

a.append(y)

In [62]:
a

[{'Unnamed: 0.1': 2213,
  'Unnamed: 0': 2213,
  'title': 'Machine Learning in Healthcare: Fundamentals & Applications',
  'enrolled': 'Enrollment number not found',
  'rating': 'Rating not found',
  'Instructor': 'Sonya Makhni',
  'Organization': 'Northeastern University',
  'Description': 'Examines data mining perspectives and methods in a healthcare context. Introduces the theoretical foundations for major data mining methods and studies how to select and use the appropriate data mining method and the major advantages for each. Students are exposed to contemporary data mining software applications and basic programming skills. Focuses on solving real-world problems, which require data cleaning, data transformation, and data modeling. In this module, we’ll start demystifying the terminology. We’ll begin by exploring the differences between AI, machine learning and deep learning. You’ll also gain hands-on experience in planning your own AI algorithm development, and learn what goes int